Com o Resultado do `fpgrowth`, descobrimos que a carta `jirachi` uma carta amplamante utilizadas entre os jogadores nos torneios de 2019.
Iremos utilizar um algoritmo SD como alvo o pokemon jirachi para encontrar combinacoes de cartas que influenciem um jogador subir no ranque em um torneio

## Bibliotecas utilizadas e carregando dataset

In [20]:
import numpy as np
import pandas as pd

df_original = pd.read_csv('./Data/tournaments.csv')
df = df_original.copy()

## Filtragem de colunas no dataset

Tirando colunas id_card, amount_card, price_card, combo_type_id, combo_type_name, id_player, region_tournament, month_tournament, day_tournament, valid_rotation_at_tournament, rotation_name, year_begin, month_begin,	day_begin.

In [21]:
columns_to_remove = [
    'id_card', 'amount_card', 'price_card', 'combo_type_id', 
    'combo_type_name', 'id_player', 
    'month_tournament', 'day_tournament', 'valid_rotation_at_tournament', 
    'rotation_name', 'year_begin', 'month_begin', 'day_begin', 'all_time_score',
]
df.drop(columns=columns_to_remove, inplace=True)
df.tail(40)

,name_card,energy_type_card,type_card,name_player,country_player,ranking_player_tournament,id_tournament,category_tournament,name_tournament,region_tournament,country_tournament,year_tournament
114251,Jolteon-GX,GX,Pokémon,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114252,Tapu Koko-GX,GX,Pokémon,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114253,Tapu Koko ♢,Lightning,Pokémon,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114254,Buzzwole,Fighting,Pokémon,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114255,Absol,Darkness,Pokémon,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114256,Volkner,NaN,Trainer,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114257,Guzma,NaN,Trainer,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114258,Lillie,NaN,Trainer,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114259,Electropower,NaN,Trainer,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018
114260,Nest Ball,NaN,Trainer,Ju Young Hong,Korea,13,154,others,Korean League Season 1,AS-OC,Korea,2018


In [19]:
df['country_tournament'].unique()

array(['United States', 'Brazil', 'Chile', 'Sweden', 'Australia',
       'Canada', 'Germany', 'Great Britain', 'Singapore', 'Japan',
       'Malaysia', 'Philippines', 'Indonesia', 'Italy', 'France', 'Spain',
       'Puerto Rico', 'Peru', 'Argentina', 'Colombia', 'Netherlands',
       'Guatemala', 'Costa Rica', 'Thailand', 'Taiwan', 'Korea'],
      dtype=object)